In [1]:
import pandas as pd
import numpy as np
import time
#from google.colab import drive
#drive.mount('/content/gdrive')


small_data_frame = pd.read_csv("CS205_small_Data__48.txt", sep=r"\s\s+", header=None, engine='python')
large_data_frame = pd.read_csv("CS205_large_Data__35.txt", sep=r"\s\s+", header=None, engine='python')



large_data_frame.columns = ['class'] + [f'F {i}' for i in range(1, large_data_frame.shape[1])]
small_data_frame.columns = ['class'] + [f'F {i}' for i in range(1, small_data_frame.shape[1])]

print(large_data_frame)
print(small_data_frame)

      class       F 1       F 2       F 3       F 4       F 5       F 6  \
0       2.0  0.032032 -0.376022 -1.367298 -1.042474 -0.345071  0.701569   
1       2.0 -0.016373 -1.135494  1.894090  0.880802  0.808286 -0.947590   
2       1.0 -0.535123  1.420478 -0.597997  0.004540  0.071119 -1.200386   
3       2.0  0.730799 -1.424075 -0.057786  1.092491 -0.301469  0.389349   
4       2.0 -0.272881  0.181582 -1.197863  0.556003 -0.875082 -1.303505   
...     ...       ...       ...       ...       ...       ...       ...   
4995    2.0 -0.126664 -0.761098 -0.985634 -0.290202 -0.415054 -0.331240   
4996    2.0 -0.686902 -1.484911 -0.058131  1.933059  1.164238 -0.113967   
4997    2.0  1.284348 -0.394168 -0.393815 -0.611538  1.453611 -0.423033   
4998    2.0 -0.762588 -0.404879 -1.317449 -0.060680 -1.384451 -0.338960   
4999    2.0  0.531829 -0.620129  0.338253 -0.674107 -0.467489 -0.118886   

           F 7       F 8       F 9  ...      F 41      F 42      F 43  \
0     0.239672 -0.670839  

In [2]:
def  normalization_of_data(data_frame):
    #Normalization of the feature columns of the dataframe
    feature_columns = data_frame.columns[1:]
    means = data_frame[feature_columns].mean()
    standard_deviation = data_frame[feature_columns].std()
    data_frame[feature_columns] = (data_frame[feature_columns] - means) / standard_deviation
    return data_frame

def find_nearest_neighbour(original_dataset_dupicate , original_dataset ,idx ) :
  squared_variance = (original_dataset_dupicate - original_dataset[idx] ) **2
  squared_range = np.add.reduce(squared_variance, axis=1)
  range = np.sqrt(squared_range)
  nearest_neighbour = np.argmin(range, keepdims=True)
  return nearest_neighbour



def calculate_accuracy(original_dataset, class_labels):

    #Calculate the accuracy using leave-one-out nearest neighbor classification
    def leave_one_out_validation(index):
        original_dataset_dupicate = np.delete(original_dataset, index, axis=0)
        nearest_index = find_nearest_neighbour(original_dataset_dupicate , original_dataset ,index )
        # print(nearest_index, index)

        if nearest_index >= index :
           nearest_index += 1

        # print(nearest_index, index)
        return class_labels[nearest_index]  ==  class_labels[index]

    correct_predictions = sum(leave_one_out_validation(index) for index in range(len(original_dataset)))
    exactness = correct_predictions / len(class_labels)
    return exactness



In [3]:
from ast import Attribute
def forward_selection(datatable):
   #Best feature set choosen by doing forward selection
    # Extract class labels
    class_labels = datatable.iloc[:, 0].to_numpy()
    names_of_feature = [index for index in datatable.columns[1:]]
    choosen_features = []
    best_features = [None, -1]

    print(f"This dataset consists of {len( names_of_feature)} features (excluding the class attribute), contains {datatable.count()[0]} instances.")
    print()
    initial_accuracy = calculate_accuracy(datatable.iloc[:,1:].to_numpy(),class_labels)
    print(f"Executing nearest neighbour on all {len( names_of_feature)} features using 'leave-one out' evaluation, results in {np.round(initial_accuracy*100, 4)}% accuracy.")
    print()
    print(" Search is Beginning ")

    mean_of_counts = 0
    sum_of_accuracy = 0
    best_accuracy = calculate_accuracy(datatable.iloc[:,1:1].to_numpy(),class_labels)
    print("Best Accuracy:", best_accuracy)
    print("Features:", names_of_feature)


    #Best feature selection
    for iteration in range(len(names_of_feature)):
      highest_accuracy = -1
      good_feature = None
      # Identify features that have not yet been selected
      features_not_selected =[feature for feature in names_of_feature if feature not in choosen_features]

      for name_of_feature in features_not_selected :
        alternate_feature = choosen_features +[name_of_feature]
        feature_matrix = datatable[ alternate_feature].to_numpy() #.reshape(-1,1) # if iteration == 0 else datatable[ alternate_selection]
        accuracy = calculate_accuracy(feature_matrix, class_labels )
        print(f"Using feature {', '.join(alternate_feature) if alternate_feature  else 'None'} gives {np.round(accuracy * 100, 4)}% accuracy")
        if accuracy > highest_accuracy:
           highest_accuracy = accuracy
           good_feature = name_of_feature

      choosen_features.append(good_feature)


      # update the higest accuracy
      if  highest_accuracy > best_accuracy:
          best_features = [choosen_features.copy()]
          best_accuracy = highest_accuracy
      else:
          print("(Alert: there is decrease in accuracy ,but we are proceeding in case this is a local maximum )")

      print(f"Features set {choosen_features } was best , and accuracy is {np.round(highest_accuracy* 100, 4)}%\n")




      # mean_of_counts += 1
      # sum_of_accuracy +=  highest_accuracy
      # mean_accuracy = sum_of_accuracy / mean_of_counts

    print(f"Hurray Completed ,the bestest feature set is {best_features}  with accuracy of {np.round(best_accuracy*100,4)}% ")
      # print()
      # print(f"Average accuracy over {mean_of_counts} iteration is {np.round(mean_accuracy*100,4) } %")

In [4]:
def backward_elimination(datatable):
   #Best feature set choosen by doing backward selection

     # Extract class labels
    class_labels = datatable.iloc[:, 0].to_numpy()

    names_of_feature = [index for index in datatable.columns[1:]]
    # Start with all features selected
    choosen_features = names_of_feature.copy()
    # To track of highest accuracy and corresponding feature set
    best_feature =[None , -1]

    #print(f"This dataset consists of {len( names_of_feature)} features (excluding the class attribute), contains {datatable.count()[0]} instances.")
    print(f"This dataset consists of {len(names_of_feature)} features (excluding the class attribute), contains {datatable.shape[0]} instances.")

    #print(f"This dataset consists of {len(names_of_feature)} features (excluding the class attribute), contains {datatable.count()[0]} instances.")

    print()
    initial_accuracy = calculate_accuracy(datatable.iloc[:,1:].to_numpy(),class_labels)
    print(f"Executing nearest neighbour on all {len( names_of_feature)} features using 'leave-one out' evaluation, results in {np.round(initial_accuracy*100, 4)}% accuracy.")
    print()
    print(" Search is Beginning ")

    mean_of_counts = 0
    sum_of_accuracy = 0
    best_accuracy = initial_accuracy

    for iteration in range(len(names_of_feature) - 1):
      highest_accuracy = -1
      bad_feature = None

      for name_of_feature in choosen_features:
        alternate_feature = choosen_features.copy()
        alternate_feature.remove(name_of_feature)
        feature_matrix = datatable[alternate_feature].to_numpy()
        accuracy = calculate_accuracy ( feature_matrix,class_labels)
        print(f"Using feature {', '.join(alternate_feature) if alternate_feature  else 'None'} gives {np.round(accuracy * 100, 4)}% accuracy")
        if accuracy >  highest_accuracy:
           highest_accuracy = accuracy
           bad_feature = name_of_feature


      choosen_features.remove(bad_feature)
      if highest_accuracy >= best_accuracy:
        # choosen_features.remove(bad_feature)
        best_feature = choosen_features[:]
        best_accuracy = highest_accuracy
      else:
         print("(Alert: there is decrease in accuracy ,but we are proceeding in case this is a local maximum )")

      print(f"Features set {choosen_features } was best , and accuracy is {np.round(highest_accuracy* 100, 4)}%")
      # else:
      #   print(f"Stopping because the accuracy is not improved.")
      #   break




    #   sum_of_accuracy +=  highest_accuracy
    #   mean_of_counts += 1

    # if highest_accuracy > best_feature [1]:
    #   best_feature = [choosen_features.copy(), highest_accuracy]
    # else:
    #   print("(Alert: there is decrease in accuracy ,but we are proceeding in case this is a local maximum )")



    # mean_accuracy = sum_of_accuracy / mean_of_counts

    print(f"Hurray Completed ,the bestest feature set is {best_feature[:]}  with accuracy of {np.round(best_accuracy*100,4)}% ")
    # print()
    # print(f"Average accuracy over {mean_of_counts} iteration is {np.round(mean_accuracy*100,4) } %")


In [5]:
def execution(choose_the_dataset = None , choose_the_algorithm = None) :

  print("Step into the Enigma of Swathi's Algorithm wonderland" )


  choices_of_dataset = {
    1:small_data_frame  ,
    2:large_data_frame
  }

  filename_dataset = {
      1:  "CS205_small_Data__48.txt" ,

      2: "CS205_large_Data__35.txt"
  }


  if choose_the_dataset is None :
    print("Kindly choose a algorithm by inputting a number :")
    for index in range(1, len (choices_of_dataset)+ 1):
      print(f"{index} {filename_dataset[index]}")


    choose_the_dataset = int(input())

    if choose_the_dataset not in choices_of_dataset :
        print("Invalid dataset choice . Please enter a valid number !")
        exit()

    data = choices_of_dataset[choose_the_dataset]

    if choose_the_algorithm is None :
      print("Kindly choose a algorithm by inputting a number :")
      print( "Carry out Forward Selection (1) ")
      print("Carry out Backward Elimination (2) ")


      choose_the_algorithm = int(input())

      if choose_the_algorithm < 1 or choose_the_algorithm > 2:
          print("Invalid algorithm choice. Please enter either 1 or 2!")
          exit()

      name_of_algorith = "Forward Selection " if choose_the_algorithm == 1 else "Backward Elimination"
      print(f"You have choosen to perform {choose_the_algorithm} on the dataset'{filename_dataset[choose_the_dataset]}'.")
      print(f"{data.shape[1]-1} features are present in the dataset (excluding the class attribute) and {data.shape[0]} instances.")

      start_time =time.time()

      if choose_the_algorithm == 1 :
       forward_selection(data)
      elif choose_the_algorithm == 2 :
       backward_elimination(data)

      stop_time = time.time()

      total_run_time= round((stop_time - start_time) , 2)

      print("Total Time " + str(total_run_time))

execution()

Step into the Enigma of Swathi's Algorithm wonderland
Kindly choose a algorithm by inputting a number :
1 CS205_small_Data__48.txt
2 CS205_large_Data__35.txt


 1


Kindly choose a algorithm by inputting a number :
Carry out Forward Selection (1) 
Carry out Backward Elimination (2) 


 1


You have choosen to perform 1 on the dataset'CS205_small_Data__48.txt'.
12 features are present in the dataset (excluding the class attribute) and 500 instances.
This dataset consists of 12 features (excluding the class attribute), contains 500 instances.

Executing nearest neighbour on all 12 features using 'leave-one out' evaluation, results in [75.8]% accuracy.

 Search is Beginning 
Best Accuracy: [0.826]
Features: ['F 1', 'F 2', 'F 3', 'F 4', 'F 5', 'F 6', 'F 7', 'F 8', 'F 9', 'F 10', 'F 11', 'F 12']
Using feature F 1 gives [73.4]% accuracy
Using feature F 2 gives [71.4]% accuracy
Using feature F 3 gives [71.4]% accuracy
Using feature F 4 gives [73.6]% accuracy
Using feature F 5 gives [71.2]% accuracy
Using feature F 6 gives [70.8]% accuracy
Using feature F 7 gives [75.2]% accuracy
Using feature F 8 gives [71.]% accuracy
Using feature F 9 gives [72.4]% accuracy


C:\Users\EndUser\AppData\Local\Temp\ipykernel_12928\2184940581.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"This dataset consists of {len( names_of_feature)} features (excluding the class attribute), contains {datatable.count()[0]} instances.")


Using feature F 10 gives [86.4]% accuracy
Using feature F 11 gives [69.4]% accuracy
Using feature F 12 gives [73.6]% accuracy
Features set ['F 10'] was best , and accuracy is [86.4]%

Using feature F 10, F 1 gives [82.6]% accuracy
Using feature F 10, F 2 gives [82.6]% accuracy
Using feature F 10, F 3 gives [80.2]% accuracy
Using feature F 10, F 4 gives [84.2]% accuracy
Using feature F 10, F 5 gives [85.2]% accuracy
Using feature F 10, F 6 gives [83.]% accuracy
Using feature F 10, F 7 gives [84.]% accuracy
Using feature F 10, F 8 gives [82.6]% accuracy
Using feature F 10, F 9 gives [83.6]% accuracy
Using feature F 10, F 11 gives [83.6]% accuracy
Using feature F 10, F 12 gives [96.4]% accuracy
Features set ['F 10', 'F 12'] was best , and accuracy is [96.4]%

Using feature F 10, F 12, F 1 gives [94.2]% accuracy
Using feature F 10, F 12, F 2 gives [94.6]% accuracy
Using feature F 10, F 12, F 3 gives [89.4]% accuracy
Using feature F 10, F 12, F 4 gives [90.2]% accuracy
Using feature F 10, F

In [6]:
choices_of_dataset = {
1:small_data_frame,
2:large_data_frame
}

filename_dataset = {
  1:  "CS205_small_Data__48.txt",

  2: "CS205_large_Data__35.txt"
}
def perform_algorithm(choose_the_dataset, choose_the_algorithm):
    start_time =time.time()
    data = choices_of_dataset[choose_the_dataset]
    if choose_the_algorithm == 1 :
        forward_selection(data)
    elif choose_the_algorithm == 2 :
        backward_elimination(data)
    
    stop_time = time.time()
    total_run_time= round((stop_time - start_time) , 2)
    
    print("Total Time " + str(total_run_time))         

In [7]:
perform_algorithm(1,1)

This dataset consists of 12 features (excluding the class attribute), contains 500 instances.

Executing nearest neighbour on all 12 features using 'leave-one out' evaluation, results in [75.8]% accuracy.

 Search is Beginning 
Best Accuracy: [0.826]
Features: ['F 1', 'F 2', 'F 3', 'F 4', 'F 5', 'F 6', 'F 7', 'F 8', 'F 9', 'F 10', 'F 11', 'F 12']
Using feature F 1 gives [73.4]% accuracy
Using feature F 2 gives [71.4]% accuracy
Using feature F 3 gives [71.4]% accuracy
Using feature F 4 gives [73.6]% accuracy
Using feature F 5 gives [71.2]% accuracy
Using feature F 6 gives [70.8]% accuracy


C:\Users\EndUser\AppData\Local\Temp\ipykernel_12928\2184940581.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"This dataset consists of {len( names_of_feature)} features (excluding the class attribute), contains {datatable.count()[0]} instances.")


Using feature F 7 gives [75.2]% accuracy
Using feature F 8 gives [71.]% accuracy
Using feature F 9 gives [72.4]% accuracy
Using feature F 10 gives [86.4]% accuracy
Using feature F 11 gives [69.4]% accuracy
Using feature F 12 gives [73.6]% accuracy
Features set ['F 10'] was best , and accuracy is [86.4]%

Using feature F 10, F 1 gives [82.6]% accuracy
Using feature F 10, F 2 gives [82.6]% accuracy
Using feature F 10, F 3 gives [80.2]% accuracy
Using feature F 10, F 4 gives [84.2]% accuracy
Using feature F 10, F 5 gives [85.2]% accuracy
Using feature F 10, F 6 gives [83.]% accuracy
Using feature F 10, F 7 gives [84.]% accuracy
Using feature F 10, F 8 gives [82.6]% accuracy
Using feature F 10, F 9 gives [83.6]% accuracy
Using feature F 10, F 11 gives [83.6]% accuracy
Using feature F 10, F 12 gives [96.4]% accuracy
Features set ['F 10', 'F 12'] was best , and accuracy is [96.4]%

Using feature F 10, F 12, F 1 gives [94.2]% accuracy
Using feature F 10, F 12, F 2 gives [94.6]% accuracy
Using

In [8]:
perform_algorithm(1,2)

This dataset consists of 12 features (excluding the class attribute), contains 500 instances.

Executing nearest neighbour on all 12 features using 'leave-one out' evaluation, results in [75.8]% accuracy.

 Search is Beginning 
Using feature F 2, F 3, F 4, F 5, F 6, F 7, F 8, F 9, F 10, F 11, F 12 gives [77.2]% accuracy
Using feature F 1, F 3, F 4, F 5, F 6, F 7, F 8, F 9, F 10, F 11, F 12 gives [76.2]% accuracy
Using feature F 1, F 2, F 4, F 5, F 6, F 7, F 8, F 9, F 10, F 11, F 12 gives [78.]% accuracy
Using feature F 1, F 2, F 3, F 5, F 6, F 7, F 8, F 9, F 10, F 11, F 12 gives [77.2]% accuracy
Using feature F 1, F 2, F 3, F 4, F 6, F 7, F 8, F 9, F 10, F 11, F 12 gives [78.]% accuracy
Using feature F 1, F 2, F 3, F 4, F 5, F 7, F 8, F 9, F 10, F 11, F 12 gives [75.4]% accuracy
Using feature F 1, F 2, F 3, F 4, F 5, F 6, F 8, F 9, F 10, F 11, F 12 gives [76.2]% accuracy
Using feature F 1, F 2, F 3, F 4, F 5, F 6, F 7, F 9, F 10, F 11, F 12 gives [77.4]% accuracy
Using feature F 1, F 2

In [9]:
perform_algorithm(2,1)

This dataset consists of 50 features (excluding the class attribute), contains 5000 instances.



C:\Users\EndUser\AppData\Local\Temp\ipykernel_12928\2184940581.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"This dataset consists of {len( names_of_feature)} features (excluding the class attribute), contains {datatable.count()[0]} instances.")


Executing nearest neighbour on all 50 features using 'leave-one out' evaluation, results in [68.44]% accuracy.

 Search is Beginning 
Best Accuracy: [0.811]
Features: ['F 1', 'F 2', 'F 3', 'F 4', 'F 5', 'F 6', 'F 7', 'F 8', 'F 9', 'F 10', 'F 11', 'F 12', 'F 13', 'F 14', 'F 15', 'F 16', 'F 17', 'F 18', 'F 19', 'F 20', 'F 21', 'F 22', 'F 23', 'F 24', 'F 25', 'F 26', 'F 27', 'F 28', 'F 29', 'F 30', 'F 31', 'F 32', 'F 33', 'F 34', 'F 35', 'F 36', 'F 37', 'F 38', 'F 39', 'F 40', 'F 41', 'F 42', 'F 43', 'F 44', 'F 45', 'F 46', 'F 47', 'F 48', 'F 49', 'F 50']
Using feature F 1 gives [69.42]% accuracy
Using feature F 2 gives [69.18]% accuracy
Using feature F 3 gives [68.94]% accuracy
Using feature F 4 gives [69.6]% accuracy
Using feature F 5 gives [83.6]% accuracy
Using feature F 6 gives [70.38]% accuracy
Using feature F 7 gives [70.36]% accuracy
Using feature F 8 gives [69.48]% accuracy
Using feature F 9 gives [68.94]% accuracy
Using feature F 10 gives [68.52]% accuracy
Using feature F 11 giv

In [10]:
perform_algorithm(2,2)

This dataset consists of 50 features (excluding the class attribute), contains 5000 instances.

Executing nearest neighbour on all 50 features using 'leave-one out' evaluation, results in [68.44]% accuracy.

 Search is Beginning 
Using feature F 2, F 3, F 4, F 5, F 6, F 7, F 8, F 9, F 10, F 11, F 12, F 13, F 14, F 15, F 16, F 17, F 18, F 19, F 20, F 21, F 22, F 23, F 24, F 25, F 26, F 27, F 28, F 29, F 30, F 31, F 32, F 33, F 34, F 35, F 36, F 37, F 38, F 39, F 40, F 41, F 42, F 43, F 44, F 45, F 46, F 47, F 48, F 49, F 50 gives [68.98]% accuracy
Using feature F 1, F 3, F 4, F 5, F 6, F 7, F 8, F 9, F 10, F 11, F 12, F 13, F 14, F 15, F 16, F 17, F 18, F 19, F 20, F 21, F 22, F 23, F 24, F 25, F 26, F 27, F 28, F 29, F 30, F 31, F 32, F 33, F 34, F 35, F 36, F 37, F 38, F 39, F 40, F 41, F 42, F 43, F 44, F 45, F 46, F 47, F 48, F 49, F 50 gives [68.54]% accuracy
Using feature F 1, F 2, F 4, F 5, F 6, F 7, F 8, F 9, F 10, F 11, F 12, F 13, F 14, F 15, F 16, F 17, F 18, F 19, F 20, F 21